Код использует модели из https://github.com/passalis/pkth

Цель ноутбука --- извлечение логитов из моделей учителя для быстрого использования в дистилляции

In [1]:
from resnet import *
from cifar_tiny import *
from cifar_dataset import *    
import torch as t 
import numpy as np

In [2]:
train_loader, test_loader, train_loader_no_augumentation = cifar10_loader(batch_size=128)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Учитель: ResNet

In [3]:
!wget https://github.com/passalis/pkth/blob/master/cifar10/models/resnet18_cifar10.model?raw=true

--2021-08-25 10:13:16--  https://github.com/passalis/pkth/blob/master/cifar10/models/resnet18_cifar10.model?raw=true
Распознаётся github.com (github.com)… 140.82.121.4
Подключение к github.com (github.com)|140.82.121.4|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 302 Found
Адрес: https://github.com/passalis/pkth/raw/master/cifar10/models/resnet18_cifar10.model [переход]
--2021-08-25 10:13:16--  https://github.com/passalis/pkth/raw/master/cifar10/models/resnet18_cifar10.model
Повторное использование соединения с github.com:443.
HTTP-запрос отправлен. Ожидание ответа… 302 Found
Адрес: https://raw.githubusercontent.com/passalis/pkth/master/cifar10/models/resnet18_cifar10.model [переход]
--2021-08-25 10:13:16--  https://raw.githubusercontent.com/passalis/pkth/master/cifar10/models/resnet18_cifar10.model
Распознаётся raw.githubusercontent.com (raw.githubusercontent.com)… 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Подключение к raw.githubusercontent.com

In [6]:
teacher = ResNet18(10).to(device)
teacher.load_state_dict(t.load('resnet18_cifar10.model?raw=true', map_location=device), )
teacher.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): PreActBlock(
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (shortcut): Sequential()
    )
    (1): PreActBlock(
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1

In [6]:
# проверим качество модели на обучении и тесте

In [7]:
correct = 0
total = 0
with t.no_grad():
    for x,y in train_loader_no_augumentation:
        x = x.to(device)
        y = y.to(device)
        out = teacher(x)
        correct += t.eq(t.argmax(out, 1), y).sum()
        total+=len(x)
print ('teacher accuracy on train', correct/total)        
        
correct = 0
total = 0
with t.no_grad():
    for x,y in test_loader:
        x = x.to(device)
        y = y.to(device)
        out = teacher(x)
        correct += t.eq(t.argmax(out, 1), y).sum()
        total+=len(x)
print ('teacher accuracy on test', correct/total)        
        
                

teacher accuracy on train tensor(0.9995, device='cuda:0')


KeyboardInterrupt: 

In [25]:
# запишем все логиты учителя

In [5]:
logits = []
with t.no_grad():
    for x,y in train_loader_no_augumentation:
        x = x.to(device)        
        out = teacher(x)
        logits.append(out.cpu().detach().numpy())
         

In [15]:
np.save('logits_resnet', np.vstack(logits), allow_pickle=False)

# Учитель: CNN

In [1]:
!wget https://github.com/passalis/pkth/blob/master/cifar10/models/aux_pkt.model?raw=true

--2021-07-18 17:40:36--  https://github.com/passalis/pkth/blob/master/cifar10/models/aux_pkt.model?raw=true
Распознаётся github.com (github.com)… 140.82.121.3
Подключение к github.com (github.com)|140.82.121.3|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 302 Found
Адрес: https://github.com/passalis/pkth/raw/master/cifar10/models/aux_pkt.model [переход]
--2021-07-18 17:40:38--  https://github.com/passalis/pkth/raw/master/cifar10/models/aux_pkt.model
Повторное использование соединения с github.com:443.
HTTP-запрос отправлен. Ожидание ответа… 302 Found
Адрес: https://raw.githubusercontent.com/passalis/pkth/master/cifar10/models/aux_pkt.model [переход]
--2021-07-18 17:40:39--  https://raw.githubusercontent.com/passalis/pkth/master/cifar10/models/aux_pkt.model
Распознаётся raw.githubusercontent.com (raw.githubusercontent.com)… 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Подключение к raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|

In [2]:
teacher = Cifar_Tiny(10).to(device)
teacher.load_state_dict(t.load('aux_pkt.model?raw=true', map_location=device), )
teacher.eval()

NameError: name 'Cifar_Tiny' is not defined

In [32]:
# проверим качество модели на обучении и тесте

In [33]:
correct = 0
total = 0
with t.no_grad():
    for x,y in train_loader_no_augumentation:
        x = x.to(device)
        y = y.to(device)
        out = teacher(x)
        correct += t.eq(t.argmax(out, 1), y).sum()
        total+=len(x)
print ('teacher accuracy on train', correct/total)        
        
correct = 0
total = 0
with t.no_grad():
    for x,y in test_loader:
        x = x.to(device)
        y = y.to(device)
        out = teacher(x)
        correct += t.eq(t.argmax(out, 1), y).sum()
        total+=len(x)
print ('teacher accuracy on test', correct/total)        
        
                

teacher accuracy on train tensor(0.7975, device='cuda:0')
teacher accuracy on test tensor(0.7767, device='cuda:0')


In [17]:
logits = []
with t.no_grad():
    for x,y in train_loader_no_augumentation:
        x = x.to(device)        
        out = teacher(x)
        logits.append(out.cpu().detach().numpy())
         

In [18]:
np.save('logits_cnn', np.vstack(logits), allow_pickle=False)